# Cohere RAG with dense retriever and ReRank model
- references: https://docs.cohere.com/v2/docs/rag-complete-example

In [1]:
import cohere
import numpy as np
from typing import List, Tuple, Dict
import os
from dotenv import load_dotenv
import json
import time # for timing functions
import sys
from colorama import Fore, Style, Back


In [2]:
# load secret from local .env file
def get_key():
    #load secret .env file
    load_dotenv()

    #store credentials
    _key = os.getenv('COHERE_API_KEY')

    #verify if it worked
    if _key is not None:
        print(Fore.GREEN + "all is good, beautiful!")
        return _key
    else:
        print(Fore.LIGHTRED_EX + "API Key is missing")

# initilize client
co = cohere.ClientV2(get_key())
        

all is good, beautiful!


In [3]:
# load documents
#read documents as .txt files in data director
def read_documents_with_doi(directory_path: str) -> List[Dict[str, str]]:
    """
    Reads documents and their DOIs from individual files in a directory.

    Args:
        directory_path (str): Path to the directory containing the document files.

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing 'doi' and 'text' keys.
    """
    global documents_with_doi
    documents_with_doi = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()
                if len(lines) >= 1:
                    doi = lines[0]
                    text = "".join(lines[1:]).strip()
                    documents_with_doi.append(f"{doi} {text}\n")
    return documents_with_doi

# initialize empty search query
search_queries = []
# Embed the documents
def document_embed(documents:List[str])->List[float]:
    """
    Embeds the documents from a list provided from read_documents_with_doi()
    """
    doc_emb = co.embed(
        model="embed-v4.0",
        input_type="search_document",
        texts=[doc for doc in documents],
        embedding_types=["float"],
        ).embeddings.float
    return doc_emb

# Embed the search query
def query_embed(search_queries:List[str])->List[float]:
    """
    Embeds the query from a list provided in search_queries variable
    """
    query_emb = co.embed(
        model="embed-v4.0",
        input_type="search_query",
        texts=search_queries,
        embedding_types=["float"],
        ).embeddings.float
    return query_emb

# retrieve top_k and compute similarity using dot product
def retrieve_top_k(top_k, query_embedded, documents_embedded, documents)->List[str]:
    """
    returns the top_k documents based on dot product similarity
    """

    scores = np.dot(query_embedded, np.transpose(documents_embedded))[0]#ordered list!
    # takes top scores, and returns sorted list and returns indices sliced by top_k
    max_idx = np.argsort(-scores)[:top_k]
    # returns documents by index
    retrieved_docs = [documents[item] for item in max_idx]
    # returns a list of documents
    return retrieved_docs

def rerank_documents(retrieved_documents,search_queries,threshold,top_k)->List[str]:
    """
    takes retrieved_documents as input along with search_queries and runs them through the 
    rerank model from cohere for semantic similarity. 

    top_n = top_k
    Limits those returned by a threshold score. this is to reduce those that are irrelevant.
    """
    # Rerank the documents
    results = co.rerank(
        model="rerank-v3.5",
        query=search_queries[0],
        documents=[doc for doc in retrieved_documents],
        top_n=top_k,
        max_tokens_per_doc=4096,# defaults to 4096
    )

    # Display the reranking results
    #for idx, result in enumerate(results.results):
    #    print(f"Rank: {idx+1}")
    #    print(f"Score: {result.relevance_score}")
    #    print(f"Document: {retrieved_documents[result.index]}\n")

    #returns only those over threshold
    reranked_docs = [
        retrieved_documents[result.index] for result in results.results if result.relevance_score >=threshold
    ]
    reranked_with_score = [(result.relevance_score, retrieved_documents[result.index].split("\n")[0].strip("DOI: ")) for result in results.results if result.relevance_score >=threshold]

    print(f"reranked_documents: {reranked_docs}")
    print(f"length of reranked_documents: {len(reranked_docs)}")

    return reranked_docs, reranked_with_score

def cohere_rag_pipeline(directory_path,search_queries,top_k,threshold):

    # retrieve documents from directory
    documents = read_documents_with_doi(directory_path)
    print(f"Length of documents: {len(documents)}")
    # embed the documents
    documents_embedded = document_embed(documents)

    #embed the query:
    query_embedded = query_embed(search_queries)

    # retrieve the top_k documents
    retrieved_documents = retrieve_top_k(top_k, query_embedded, documents_embedded, documents)

    # rerank the documents using the Rerank model from Cohere
    reranked_documents, reranked_DOIs_with_score = rerank_documents(retrieved_documents,search_queries,threshold,top_k)
    # set system instructions
    instructions = """
                    You are an academic research assistant.
                    You must include the DOI in your response.
                    If there is no content provided, ask for a different question.
                    Please structure your response like this:
                    Summary: summary statement here. 
                    DOI: summary of the text associated with this DOI.
                    Address me as, 'my lady'.
                    """
    # create messages to model
    messages = [{"role":"user",
                "content": search_queries[0]},
                {"role":"system",
                "content":instructions}]

    # Generate the response
    resp = co.chat(
        model="command-a-03-2025",
        messages=messages,
        documents=reranked_documents,
    )

    return resp, reranked_documents, reranked_DOIs_with_score

# ****** Pipeline ********
# set directory path
directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data"
# initialize search_queries 
search_queries = [input("what is your query?")]#could be a list of multiple queries
# set top_k
top_k = 5
#set threshold 
threshold = 0.1

response, reranked_documents_end, reranked_DOIs_with_score_end = cohere_rag_pipeline(directory_path,search_queries,top_k,threshold)
# Display the response
print(response.message.content[0].text)
print(Fore.LIGHTCYAN_EX + f"------\nReranked documents:")
for doc in reranked_documents_end:
    print(doc)

# Display the citations and source documents
if response.message.citations:
    print(Fore.LIGHTYELLOW_EX + "\nCITATIONS:")
    for citation in response.message.citations:
        print(f"source text: {citation.text},\nsource: {citation.sources[0].document.get('content').split("\n")[0]}\n------")


Length of documents: 45
reranked_documents: ['DOI: 10.1162/qss_a_00286\n Title: Completeness degree of publication metadata in eight free-access scholarly databases\nAbstract: Abstract The main objective of this study is to compare the amount of metadata and the completeness degree of research publications in new academic databases. Using a quantitative approach, we selected a random Crossref sample of more than 115,000 records, which was then searched in seven databases (Dimensions, Google Scholar, Microsoft Academic, OpenAlex, Scilit, Semantic Scholar, and The Lens). Seven characteristics were analyzed (abstract, access, bibliographic info, document type, publication date, language, and identifiers), to observe fields that describe this information, the completeness rate of these fields, and the agreement among databases. The results show that academic search engines (Google Scholar, Microsoft Academic, and Semantic Scholar) gather less information and have a low degree of completene

In [5]:
reranked_DOIs_with_score_end

[(0.6519982, '10.1162/qss_a_00286'),
 (0.3312825, '10.48550/arXiv.2303.17661'),
 (0.32875618, '10.31222/osf.io/smxe5'),
 (0.19046825, '10.31274/b8136f97.ccc3dae4'),
 (0.10833058, '10.1002/leap.1411')]

# Analysis
Precision, recall, accuracy, F1 scores and faithfulness
## Precision, recall, F1 score
### references
- https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
- https://vitalflux.com/accuracy-precision-recall-f1-score-python-example/


In [6]:
from typing import List, Dict, Set
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
import openpyxl
import pandas as pd

In [106]:
#initial dataframe to capture results from each query and results
#ONLY DO THIS AT THE BEGINNING OF THE ANALYSIS PROCEDURE, OTHERWISE, IT WILL ERASE THE PREVIOUS RESULTS!!

results_df = pd.DataFrame(columns=['Query','Precision','Recall','F1-Score','Accuracy', 'Balanced accuracy', 'Faithfulness score', 'Documents score', 'Response'])
results_df

,Query,Precision,Recall,F1-Score,Accuracy,Balanced accuracy,Faithfulness score,Documents score,Response


## set up functions

In [107]:
from typing import List, Dict
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
"""
change this to read in an excel sheet of queries and ground_truth dois.
Then it should be isolated as a function.
Run the function to iterature through the list.
"""

# Extract DOIs from retrieved documents
retrieved_dois = [doc.split("\n")[0].strip("DOI: ") for doc in reranked_documents_end]
print("Retrieved DOIs:", retrieved_dois)

# initiates the variable
ground_truth = []

def evaluate_retrieval(retrieved_dois, ground_truth):
    corpus_doi_list = []
    #corpus_list is a global variable in rag_pipeline()
    for each in range(len(documents_with_doi)):
        a = documents_with_doi[each].split("\n")[0].strip("DOI: ")
        corpus_doi_list.append(a)
    print(len(corpus_doi_list))

    def compare_lists(list1, list2, list3):
        for val in list1:
            if val in list2:
                list3.append(1)
            else:
                list3.append(0)

    #set y_true so that len(y_true)==len(corpus_doi_list)
    y_true = []
    compare_lists(corpus_doi_list,ground_truth,y_true)
    y_true = np.array(y_true)
    y_pred = []
    compare_lists(corpus_doi_list,retrieved_dois,y_pred)
    y_pred = np.array(y_pred)

    # calculate metrics - could also use sklearn.metrics functions such as precision_score, but this is easier to read
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average="micro")
    accuracy = accuracy_score(y_true, y_pred, normalize=True)
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

    faithfulness_score = 0
    for each in retrieved_dois:
        if each in response.message.content[0].text:
            faithfulness_score+=1
        else:
            faithfulness_score+=0

    return {
        'Query':f"{search_queries[0]}",
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy":accuracy,
        "Balanced accuracy":balanced_accuracy,
        "Faithfulness score":faithfulness_score,
        "Documents score":str(reranked_DOIs_with_score_end),#converted to string because pandas was only taking the first tuple not the entire list - use ast.literal_eval() to unpack later.
        "Response":response.message.content[0].text
    }

def print_results()->Dict:
    """
    Prints a nicely ordered set of results from evalaute_retrieval()
    """
    results = evaluate_retrieval(retrieved_dois, ground_truth)
    print(f"For query: {results['Query']}:")
    print(f"Precision: {results['Precision']:.3f}")
    print(f"Recall: {results['Recall']:.3f}")
    print(f"F1-Score: {results['F1-Score']:.3f}")
    print(f"Accuracy: {results['Accuracy']:.3f}")
    print(f"Balanced accuracy: {results['Balanced accuracy']:.3f}")
    print(f"Faithfulness score: {results['Faithfulness score']}")
    print(Fore.LIGHTMAGENTA_EX + f"Documents scores: {results['Documents score']}")
    return results

#for debugging

#print_results()


Retrieved DOIs: ['10.1162/qss_a_00286', '10.48550/arXiv.2303.17661', '10.31222/osf.io/smxe5', '10.31274/b8136f97.ccc3dae4', '10.1002/leap.1411']


## run the test from here

In [108]:
#run the test from here


# Extract DOIs from retrieved documents
retrieved_dois = [doc.split("\n")[0].strip("DOI: ") for doc in reranked_documents_end]
print("Retrieved DOIs:", retrieved_dois)

# Ground truth relevant documents (DOIs) for each query
ground_truth = ["10.1002/leap.1411","10.1007/s11192-020-03632-0","10.1162/qss_a_00286","10.1162/qss_a_00022","10.31222/osf.io/smxe5"]

#***** Begin chat session *****
# set directory path
directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data"
# initialize search_queries 
search_queries = [input("what is your query?")]#could be a list of multiple queries

# set top_k
top_k = 5
#set threshold 
threshold = 0.10

response, reranked_documents_end, reranked_DOIs_with_score_end = cohere_rag_pipeline(directory_path,search_queries,top_k,threshold)
# Display the response
print(Fore.LIGHTYELLOW_EX + f"{response.message.content[0].text}")

new_result = print_results()
# add the new result to the df
results_df.loc[len(results_df)] = new_result

#save the queries and responses to separate dataframe to be manually annontated
answer_relevance_df = results_df[['Query','Response']].copy(deep=True)

# save out answer_relevance_df
filename="analysis/dense_answer_relevance_results.xlsx"
answer_relevance_df.to_excel(filename)

filename = "analysis/dense_analysis_results.xlsx"
os.makedirs(os.path.dirname(filename), exist_ok=True)
results_df.to_excel(filename)
results_df.tail(5)

Retrieved DOIs: ['10.1162/qss_a_00286', '10.48550/arXiv.2303.17661', '10.31222/osf.io/smxe5', '10.31274/b8136f97.ccc3dae4', '10.1002/leap.1411']
Length of documents: 45
reranked_documents: ['DOI: 10.1162/qss_a_00286\n Title: Completeness degree of publication metadata in eight free-access scholarly databases\nAbstract: Abstract The main objective of this study is to compare the amount of metadata and the completeness degree of research publications in new academic databases. Using a quantitative approach, we selected a random Crossref sample of more than 115,000 records, which was then searched in seven databases (Dimensions, Google Scholar, Microsoft Academic, OpenAlex, Scilit, Semantic Scholar, and The Lens). Seven characteristics were analyzed (abstract, access, bibliographic info, document type, publication date, language, and identifiers), to observe fields that describe this information, the completeness rate of these fields, and the agreement among databases. The results show th

,Query,Precision,Recall,F1-Score,Accuracy,Balanced accuracy,Faithfulness score,Documents score,Response
0,which studies examined the abstract in metadata?,0.6,0.6,0.911111,0.911111,0.775,4,"[(0.6519982, '10.1162/qss_a_00286'), (0.331574...",Summary: Four studies examined the abstract in...
